# Keolis delays
*Russell Goldenberg*

On the format: each question is followed by the code that generates the answer. This is also known as [reproducible research](https://en.wikipedia.org/wiki/Reproducibility#Reproducible_research), a practice that’s slowly being adopted by newspapers (e.g. 538, The Upshot).


In [30]:
import re
import os
import agate

In [31]:
inputFile = os.getcwd() + '/delays.txt';
lines = open(inputFile).readlines()

In [32]:
rows = []

def check_missing(date, time):
    missing = [('12/18/15', '10:37 am'), ('12/23/15', '5:58 am')]
    for m in missing:
        if date == m[0] and time == m[1]:
            return True
    
    return False
        
def line_to_list(arr): 
    date = arr[0]
    time = arr[1] + ' ' + arr[2]
    id = arr[5]
    minutes = arr[6]
    
    if check_missing(date, time):
        id = arr[4]
        minutes = arr[5]

    return (date, time, date + ' ' + time, id, minutes)

def is_valid_line(str):
    has_date = re.search(r'(\d+/\d+/\d+)', str)
    has_time = re.search(r'(\d+:\d+)', str)
    if has_date and has_time and has_date.start() == 0:
        return True
    else: 
        return False
        
for line in lines:
    trim = line.strip()
    split = re.split(' {1,}', trim)

    if is_valid_line(trim) and len(split) > 2:
        new_data = line_to_list(split)
        rows.append(new_data)

col_types = [agate.Text(), agate.Text(), agate.DateTime(), agate.Text(), agate.Number()]
col_names = ['date', 'time', 'datetime', 'id', 'minutes']
data = agate.Table(rows, column_names=col_names, column_types=col_types)

In [33]:
print('Here is a sample of what the clean data looks like:')
data.to_csv('test.csv')
data.print_table(max_rows=5)

Here is a sample of what the clean data looks like:
|---------+----------+---------------------+------+----------|
|  date   | time     |            datetime | id   | minutes  |
|---------+----------+---------------------+------+----------|
|  6/1/15 | 6:00 am  | 2015-06-01 06:00:00 | 2034 |      18  |
|  6/1/15 | 7:20 am  | 2015-06-01 07:20:00 | 1059 |      34  |
|  6/1/15 | 7:20 am  | 2015-06-01 07:20:00 | 1059 |      34  |
|  6/1/15 | 8:40 am  | 2015-06-01 08:40:00 | 010  |      15  |
|  6/1/15 | 10:39 am | 2015-06-01 10:39:00 | 2012 |      20  |
|  ...    | ...      |                 ... | ...  |     ...  |
|---------+----------+---------------------+------+----------|


In [34]:
# helper functions
def to_percent(n, d):
    return str(round(n/d * 100, 1)) + '%'

### How many delays involved new trains?

In [35]:
# starts with 2 and >= 4 digits
with_new_trains = data.where(lambda row: row['id'].startswith('2') and len(row['id']) > 3)

In [36]:
total = len(data.rows)
total_new_trains = len(with_new_trains.rows)
percent_new_trains = to_percent(total_new_trains, total)

In [37]:
min_date = data.aggregate(agate.Min('datetime'))
max_date = data.aggregate(agate.Max('datetime'))
min_date_display= min_date.date()
max_date_display= max_date.date()

In [38]:
print('This data looks at delays from %s to %s' % (min_date_display, max_date_display))
print('There were %d total delays' % total)
print('%d delays were with new trains' % total_new_trains)
print('%s of all delays involved new trains' % percent_new_trains)

This data looks at delays from 2015-06-01 to 2015-12-31
There were 961 total delays
305 delays were with new trains
31.7% of all delays involved new trains


### How many days had a delay with a new train?

In [39]:
total_days = len(data.distinct('date').rows)
days_with_new_train = len(with_new_trains.distinct('date').rows)
percent_days_new_train = to_percent(days_with_new_train, total_days)

In [40]:
print('There were delays on %d days' % total_days)
print('On %d of those days there was at least one delay involving a new train' % days_with_new_train)
print('%s of all days with a delay involved a new train' % percent_days_new_train)

There were delays on 201 days
On 130 of those days there was at least one delay involving a new train
64.7% of all days with a delay involved a new train


### How many new unique new trains had issues?

In [41]:
unique_new_trains = len(with_new_trains.distinct('id').rows)

In [42]:
print('%d different new trains were involved in one or more delays' % unique_new_trains)

39 different new trains were involved in one or more delays


### What is the average delay?

In [43]:
median_delay = data.aggregate(agate.Median('minutes'))
print('The median delay was %d minutes' % median_delay)

The median delay was 9 minutes


### How many delays per day on average?

In [44]:
counts_date = data.counts('date')
per_day = counts_date.aggregate(agate.Mean('count'))
print('On average there were %d delays per day' % per_day)

On average there were 4 delays per day


### What was December like?

In [50]:
with_dec = data.where(lambda row: row['datetime'].month == 12)
with_dec_new_trains = with_new_trains.where(lambda row: row['datetime'].month == 12)
total_dec = len(with_dec.rows)
total_dec_new_trains = len(with_dec_new_trains.rows)
percent_dec_new_trains = to_percent(total_dec_new_trains, total_dec)

In [51]:
print('There were %d total delays' % total_dec)
print('%d delays were with new trains' % total_dec_new_trains)
print('%s of all delays involved new trains' % percent_dec_new_trains)

There were 105 total delays
31 delays were with new trains
29.5% of all delays involved new trains
